In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from functools import partial
import pickle as pkl
import parallel
import matplotlib.pyplot as plt
from dbn.models import SupervisedDBNRegression
from elm.elm import Model as elm
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.base import clone
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import ParameterGrid
import sklearn.metrics as metrics
from sklearn import preprocessing
from statsmodels.tsa.stattools import adfuller
from multiprocessing import Pool
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
from dm_test.dm_test import dm_test as dm

In [2]:
file_names = ["amz", "APPLE", "electricity", "goldman", "msft", "pollutions", "star", "sunspot", "vehicle", "wine"]
file_prefix = "dataset/"
file_suffix = ".txt"
models = [
          {'model_name': "SVR", 'windows': True, 'lags': 20, 'normalize_before_metrics': True, 'differencing_order': 'ADF', 'level': '5%', 'p_level': 0.05},
          {'model_name': "GB", 'windows': True, 'lags': 20, 'normalize_before_metrics': True, 'differencing_order': 'ADF', 'level': '5%', 'p_level': 0.05},
          {'model_name': "RF", 'windows': True, 'lags': 20, 'normalize_before_metrics': True, 'differencing_order': 'ADF', 'level': '5%', 'p_level': 0.05},
          {'model_name': "ARIMA", 'windows': False, 'lags': 20, 'normalize_before_metrics': True, 'differencing_order': 'ADF', 'level': '5%', 'p_level': 0.05},
          {'model_name': "DBN", 'windows': True, 'lags': 20, 'normalize_before_metrics': True, 'differencing_order': 'ADF', 'level': '5%', 'p_level': 0.05},
          {'model_name': "MLP", 'windows': True, 'lags': 20, 'normalize_before_metrics': True, 'differencing_order': 'ADF', 'level': '5%', 'p_level': 0.05},
          {'model_name': "ELM", 'windows': True, 'lags': 20, 'normalize_before_metrics': True, 'differencing_order': 'ADF', 'level': '5%', 'p_level': 0.05},
         ]

#431
random_seed = 5412
execution = "dsnaw" # "oracle" or "dsnaw"
load_saved_model_results = True
save_model_results = True
n_mode = "dynamic" # "static" or "dynamic" 
plot_curves = False
train_proportion = 1/2
val_proportion = 1/4
max_k = 20
max_n = len(models)

In [3]:
#read files
def read_file(path):
    array_ts = np.loadtxt(path)
    df = pd.DataFrame(data = array_ts, columns = ['y'])
    return df

In [4]:
#define split points

def define_split_points(df, train, val):
    val_index = int(df.shape[0]*train)
    test_index = int(df.shape[0]*(train+val))

    # fig, ax=plt.subplots(figsize=(9, 4))
    # df['y'].loc[:val_index-1].plot(ax=ax, label = 'train')
    # df['y'].loc[val_index:test_index-1].plot(ax=ax, label = 'validation')
    # df['y'].loc[test_index:].plot(ax=ax, label = 'test')
    # ax.legend()
    
    return val_index, test_index


In [5]:
#difference time series

def check_ADF(ts, lvl, p_lvl):
    result = adfuller(ts)
    return result[0] < result[4][lvl] and result[1] < p_lvl

def difference(df, differencing_order, level, p_level):

    df_differenced = None
    if differencing_order:    
        df_differenced = df.copy()

        if differencing_order == 'ADF':
            stationary = check_ADF(df_differenced['y'].values, level, p_level)
            d = 0
            while not stationary:
                df_differenced = df_differenced.diff().dropna()
                d += 1
                stationary = check_ADF(df_differenced['y'].values, level, p_level)
            if d == 0:
                df_differenced = None
            differencing_order = d
        else:
            i = 0
            while i < differencing_order:
                df_differenced = df_differenced.diff().dropna()
                i+=1
                
    print("d: ", differencing_order)
    # if differencing_order:
    #     fig, ax=plt.subplots(figsize=(9, 4))
    #     df_differenced.y.plot(ax = ax)
        
    return df_differenced, differencing_order


In [6]:
#normalize time series

def normalize(df, val_index, d):
    min_max_scaler = preprocessing.MinMaxScaler()
    min_max_scaler.fit(df['y'].loc[:val_index-1].values.reshape(-1,1))
    df_normalized = pd.DataFrame({'y': np.append(np.zeros(d),min_max_scaler.transform(df['y'].values.reshape(-1, 1)).flatten())}).iloc[d:]
    
    return df_normalized, min_max_scaler


In [7]:
#create windows

def create_windows(df, lags):
    df_windowed = df.copy()
    df_windowed['x'] = df_windowed['y'].shift()
    for i in range(1, lags):
        df_windowed[f'x-{i}'] = df_windowed['x'].shift(i)
    df_windowed = df_windowed.dropna()
    return df_windowed

In [8]:
#gridsearch

def parallel_gridsearch(estimator, param_grid, x_train, y_train, x_val, y_val):
    pool = Pool()
    list_params = list(ParameterGrid(param_grid))
    results = list(tqdm(pool.imap(partial(parallel.validate_params, estimator=estimator, x_train=x_train, 
                               y_train=y_train, x_val=x_val, y_val=y_val), list_params), total=len(list_params)))
    idx = np.argmin([r[1] for r in results])
    best_params = results[idx][0]
    best_rmse = results[idx][1]
    
    pool.close()
    pool.join()
    print('Best params: ', best_params)
    print('Best rmse: ', best_rmse)
    return best_params
    

In [9]:
#SVR

def svr_model(lags):
    parameters = {'C':[1, 10, 100, 1000], 'gamma': [0.001, 0.01, 0.1, 1],
                      'kernel':["rbf"],
                      'epsilon': [0.1, 0.01, 0.001, 0.0001, 0.00001],
                 }

    model = SVR(max_iter = 10000)

    return model, parameters

In [10]:
#Gradient Boosting

def gb_model(lags):
    parameters = {'n_estimators': [50, 100, 200], 
                  'max_depth': [5, 10, 15],
                  'max_features': [0.6, 0.8, 1],
                  'subsample' : [0.6, 0.8, 1],
                  'learning_rate': [0.1, 0.3, 0.5],
                 }
    
    model = GradientBoostingRegressor(random_state=random_seed)

    return model, parameters

In [11]:
#Random Forest

def rf_model(lags):
    parameters = {'n_estimators': [50, 100, 200], 
                  'max_depth': [5, 10, 15],
                  'max_features': [0.6, 0.8, 1],
                 }
    
    model = RandomForestRegressor(random_state=random_seed)

    return model, parameters

In [12]:
#MLP

def mlp_model(lags):
    parameters = {'hidden_layer_sizes': [20, 50, 100], 
                      'max_iter': [1000],
                      'tol': [0.001, 0.0001, 0.00001],
                 }
    
    model = MLPRegressor(activation='logistic', solver='lbfgs', random_state=random_seed)
    
    return model, parameters

In [13]:
#ELM

def elm_model(lags):
    parameters = {'hidden_dim': [20, 50, 100, 200, 500],  
                 }
    
    model = elm(input_dim=lags)
    
    return model, parameters

In [14]:
#Deep Belief Network

def dbn_model(lags):
    
    model = SupervisedDBNRegression(hidden_layers_structure=[100],
                                    learning_rate_rbm=0.01,
                                    learning_rate=0.01,
                                    n_epochs_rbm=20,
                                    n_iter_backprop=200,
                                    batch_size=16,
                                    activation_function='relu',
                                    verbose=False)

    return model

In [15]:
models_name = {
    'SVR': svr_model, 
    'GB': gb_model,
    'RF': rf_model,
    'DBN': dbn_model,
    'MLP': mlp_model,
    'ELM': elm_model,
         }

In [16]:
#train model
@ignore_warnings(category=ConvergenceWarning)
def train_model(model_name, df, val_index, test_index, lags):

    if model_name == "ARIMA":
        
        model = pm.auto_arima(df['y'].loc[:test_index-1],
                          test='adf',       
                          max_p=lags, max_q=lags,        
                          seasonal=False,   
                          trace=True,
                          error_action='ignore',  
                          suppress_warnings=True, 
                          stepwise=True)
        best_params=model.get_params()
        
    elif model_name == "DBN":
        
        model = dbn_model(lags)
        
        x_train = df.drop(columns = ['y']).loc[:test_index-1].values
        y_train = df['y'].loc[:test_index-1].values
        
        model.fit(x_train, y_train)
        best_params={}
        
    else:
        
        model, param_grid = models_name[model_name](lags)

        x_train = df.drop(columns = ['y']).loc[:val_index-1].values
        y_train = df['y'].loc[:val_index-1].values
        x_val = df.drop(columns = ['y']).loc[val_index:test_index-1].values
        y_val = df['y'].loc[val_index:test_index-1].values
        
        
        best_params = parallel_gridsearch(model, param_grid, 
                                    x_train, y_train,
                                    x_val, y_val)

        model.set_params(**best_params)
        model.fit(x_train, y_train)

        
    
    return model, best_params

In [17]:
#test model

def test_model(model, model_name, df, test_index, scaler):
    if model_name != "ARIMA":
        y_test = df['y'].loc[test_index:].values
        x_test = df.drop(columns = ['y']).loc[test_index:].values

        y_pred = model.predict(x_test)

    else:
        y_test = df['y'].loc[test_index:].values
        y_pred = np.zeros(y_test.size)
        for i in tqdm(range(y_pred.size)):
            y_pred[i] = model.predict(n_periods = 1)
            model.update([y_test[i]])

    y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

    return y_pred, y_test

In [18]:
def recursive_differencing_coefficients(order):
    if order <= 1:
        return np.array([1, -1])
    else:
        prev_coefficients = recursive_differencing_coefficients(order-1)
        coefficients = np.zeros(order+1)
        coefficients[0] = prev_coefficients[0]
        coefficients[-1] = -prev_coefficients[-1]
        for i in range(1, order):
            coefficients[i] = prev_coefficients[i] - prev_coefficients[i-1]
        return coefficients   

def inverse_difference(differencing_order, df, df_diff, test_index, y_pred):
    df_pred = df.copy()
    
    for i in range(1, differencing_order + 1):
        df_pred[f'y-{i}'] = df_pred['y'].shift(i)
    
    df_pred['y_diff'] = df_diff['y']
    df_pred['y_diff'][test_index:] = y_pred
    
    df_pred.fillna(0, inplace=True)
    
    coefficients = recursive_differencing_coefficients(differencing_order)
    df_pred['y_pred'] = df_pred['y_diff']
    for i in range(1, differencing_order + 1):
        df_pred['y_pred'] = df_pred['y_pred'] - coefficients[i]*df_pred[f'y-{i}']
        
    y_pred = df_pred['y_pred'][test_index:].values    
    y_test = df_pred['y'][test_index:].values
    
    return y_pred, y_test


In [19]:
#plot results
def plot_results(y_pred, y_test):
    fig, ax=plt.subplots(figsize=(9, 4))
    plt.plot(y_pred, label = 'predicted')
    plt.plot(y_test, label = 'real')
    ax.legend()
    plt.show()

In [20]:
def oracle(preds, real):
    best_pred = np.zeros(real.size)
    for i in range(real.size):
        candidates = np.array([p[i] for p in preds])
        idx = np.argmin(np.absolute(candidates - real[i]))
        best_pred[i] = candidates[idx]
    return best_pred

In [21]:
#calculate metrics

def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)

    posi_with_zeros = np.where(y_true == 0)[0]

    y_true = [n for k, n in enumerate(y_true) if k not in posi_with_zeros]
    y_pred = [n for k, n in enumerate(y_pred) if k not in posi_with_zeros]
    
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)

    posi_with_zeros = np.where(y_true == 0)[0]

    y_true = [n for k, n in enumerate(y_true) if k not in posi_with_zeros]
    y_pred = [n for k, n in enumerate(y_pred) if k not in posi_with_zeros]
    
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)

    posi_with_zeros = np.where((np.abs(y_true)+np.abs(y_pred))/2 == 0 )[0]

    y_true = [n for k, n in enumerate(y_true) if k not in posi_with_zeros]
    y_pred = [n for k, n in enumerate(y_pred) if k not in posi_with_zeros]
    
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    return np.mean(np.abs(y_true - y_pred) / ((np.abs(y_true)+np.abs(y_pred))/2)) * 100


def mean_absolute_error(y_true, y_pred):
    
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)

    return np.mean(np.abs(y_true - y_pred))

def average_relative_variance(y_true, y_pred):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    mean = np.mean(y_true)

    error_sup = np.square(np.subtract(y_true, y_pred)).sum()
    error_inf = np.square(np.subtract(y_pred, mean)).sum()

    return error_sup / error_inf

def calculate_metrics(y_pred, y_test, normalize_before_metrics, scaler, df):
    if normalize_before_metrics:
        scaler.fit(df['y'].values.reshape(-1,1))
        y_pred = scaler.transform(y_pred.reshape(-1, 1)).flatten()
        y_test = scaler.transform(y_test.reshape(-1, 1)).flatten()

    mse = metrics.mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    nrmse = rmse/(y_test.max()-y_test.min())
    mape = mean_absolute_percentage_error(y_test, y_pred)
    smape = symmetric_mean_absolute_percentage_error(y_test, y_pred)
    arv = average_relative_variance(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    
    res = {'MSE': mse, 'MAPE': mape, 'ARV': arv, 'MAE': mae, 'RMSE': rmse, 'NRMSE': nrmse, 'SMAPE': smape}
    
    # for key, value in res.items():
    #     print(f'{key}: {value}')
    
    return res

In [22]:
#gridsearch

def parallel_gridsearch_dsnaw(n_mode, df, pred_results, val_index, test_index, param_grid):
    pool = Pool()
    list_params = list(ParameterGrid(param_grid))
    results = list(tqdm(pool.imap(partial(parallel.validate_ds_params, n_mode=n_mode, df=df, pred_results=pred_results, val_index=val_index, test_index=test_index), list_params), total=len(list_params)))
    idx = np.argmin([r[1] for r in results])
    
    best_params = results[idx][0]
    best_rmse = results[idx][1]
    
    pool.close()
    pool.join()
    
    print('Best params: ', best_params)
    print('Best rmse: ', best_rmse)
    return best_params

In [23]:
def test_dsnaw(params, df, pred_results, test_index, n_mode):
    k = params['k']
    comb = params['comb']    
    
    y_dsnaw = np.zeros(df['y'].size-test_index)
    for i in range(test_index, df['y'].size):
        rmse_roc = []
        real_roc = df['y'].loc[i-k:i-1].values
        
        roc_list = []
        for j in range(0, len(pred_results)):      
            pred_roc = pred_results[j]['y'].loc[i-k:i-1].values
            roc_list.append(pred_roc)
            rmse_roc.append((j, np.sqrt(metrics.mean_squared_error(pred_roc, real_roc))))
        sorted_rmse_roc = sorted(rmse_roc, key=lambda x: x[1])
        
        roc_list_sorted = np.zeros((len(pred_results), k))
        for j in range(0, len(pred_results)):
            roc_list_sorted[j] = roc_list[sorted_rmse_roc[j][0]]        
        roc_list = roc_list_sorted
        
        if n_mode == "dynamic":
            Ns = [1, 3, 5]            
        else:
            Ns = [params['n']]
            
        n_results = [] 
        for n in Ns:
            comb_roc = np.zeros(roc_list.shape[1])
            for j in range(comb_roc.size):
                if comb == 'median':
                    comb_roc[j] = np.median(roc_list[:n, j])
                else:
                    comb_roc[j] = np.average(roc_list[:n, j])
            n_results.append(np.sqrt(metrics.mean_squared_error(comb_roc, real_roc)))
        n = Ns[np.argmin(n_results)]
        
        comb_values = np.zeros(n)
        for j in range(0, n):
            comb_values[j] = pred_results[sorted_rmse_roc[j][0]]['y'].loc[i]
        if comb == 'median':
            y_dsnaw[i-test_index] = np.median(comb_values)
        else:
            y_dsnaw[i-test_index] = np.average(comb_values)

    return y_dsnaw

In [24]:
def run_model(execution, df, model_name, train_proportion, val_proportion, windows, lags, max_k, plot_curves, normalize_before_metrics, differencing_order, level, p_level):
    print("\n###############################################\n")
    print("Executing: ", model_name)
    val_index, test_index = define_split_points(df, train_proportion, val_proportion)
    d = 0
    if differencing_order:
        df_differenced, d = difference(df, differencing_order, level, p_level)
    if(d):
        df_normalized, min_max_scaler = normalize(df_differenced, val_index, d)
    else:
        df_normalized, min_max_scaler = normalize(df, val_index, d)
    
    if(windows):
        df_normalized = create_windows(df_normalized, lags)        

    estimator, params = train_model(model_name, df_normalized, val_index, test_index, lags)
    if(execution == "dsnaw"):
        result_index = val_index - max_k
    else:
        result_index = test_index
    y_pred, y_test = test_model(estimator, model_name, df_normalized, result_index, min_max_scaler)
    if d:
        y_pred, y_test = inverse_difference(d, df, df_differenced, result_index, y_pred)        
    if(execution == "dsnaw"):
        if plot_curves:
            plot_results(y_pred[(test_index-result_index):], y_test[(test_index-result_index):])
        m = calculate_metrics(y_pred[(test_index-result_index):], y_test[(test_index-result_index):], normalize_before_metrics, min_max_scaler, df)
    else:
        if plot_curves:
            plot_results(y_pred, y_test)
        m = calculate_metrics(y_pred, y_test, normalize_before_metrics, min_max_scaler, df)
    return y_pred, m

In [25]:
metric_res = []
params_res = []
value_res = {}


for f in file_names:
    print("Time Series: ", f)
    df = read_file(file_prefix + f + file_suffix)
    
    results = []
    m_results = []

    if load_saved_model_results:
        with open('results_pkl/models_results/' + f + '.pkl', 'rb') as handle:
            results, m_results = pkl.load(handle)
    
    else:

        for args in models:
            run_result = run_model(execution=execution, df=df, train_proportion=train_proportion, val_proportion=val_proportion, plot_curves=plot_curves, max_k = max_k, **args)
            results.append((args['model_name'], run_result[0]))
            m_results.append((args['model_name'], run_result[1]))
        
        if save_model_results:
            with open('results_pkl/models_results/' + f + '.pkl', 'wb') as handle:
                pkl.dump((results, m_results), handle)

    val_index, test_index = define_split_points(df, train_proportion, val_proportion)
    min_max_scaler = preprocessing.MinMaxScaler()
    
    real = df['y'].loc[test_index:].values

    if (execution == "oracle"):
        oracle_pred = oracle([r[1] for r in results], real)
        m = calculate_metrics(oracle_pred, real, True, min_max_scaler, df)
        metric_res.append((f, m))

    else:
        
        pred_results = []
        v_results = {}
        
        
        min_max_scaler.fit(df['y'].values.reshape(-1,1))
        
        for r in results:
            temp = pd.DataFrame(data = np.zeros(df['y'].size), columns = ['y'])
            temp['y'][val_index - max_k:] = r[1]
            pred_results.append(temp)
            
            v_results[r[0]] = min_max_scaler.transform(r[1][-(df['y'].size-test_index):].reshape(-1, 1)).flatten() 
        v_results['Real'] = min_max_scaler.transform(df['y'].loc[test_index:].values.reshape(-1, 1)).flatten() 
        
        parameters = {
            'k': list(range(1,max_k +1)),
            'comb': ['median', 'average'],
        }
        if n_mode == "static":
            parameters['n'] = list(range(1,max_n +1))
            
        grid = list(ParameterGrid(parameters))
        best_params = parallel_gridsearch_dsnaw(n_mode, df, pred_results, val_index, test_index, parameters)
        params_res.append((f, best_params))
                
        y_dsnaw = test_dsnaw(best_params, df, pred_results, test_index, n_mode)
        m = calculate_metrics(y_dsnaw, real, True, min_max_scaler, df)
        m_results.append(("Proposed", m))
        metric_res.append((f, m_results))
        v_results['Proposed'] = min_max_scaler.transform(y_dsnaw.reshape(-1, 1)).flatten() 
        value_res[f] = v_results
        
    

Time Series:  amz


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:07<00:00,  5.58it/s]


Best params:  {'comb': 'median', 'k': 4}
Best rmse:  574.9834621983266
Time Series:  APPLE


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:07<00:00,  5.41it/s]


Best params:  {'comb': 'average', 'k': 7}
Best rmse:  815.0117709468019
Time Series:  electricity


...

Best params:  {'comb': 'median', 'k': 20}
Best rmse:  12.072038624611512
Time Series:  vehicle


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:02<00:00, 18.74it/s]


Best params:  {'comb': 'median', 'k': 7}
Best rmse:  0.8591232125749121
Time Series:  wine


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 20.43it/s]


Best params:  {'comb': 'average', 'k': 20}
Best rmse:  20.390818143265804


In [26]:
if execution == "dsnaw":
    dict_results = {'metric': metric_res, 'params': params_res, 'value': value_res}
    if n_mode == "dynamic":
        dy_str = "_dynamic_n"
    else:
        dy_str = ""
    with open('results_pkl/proposed' + dy_str + '.pkl', 'wb') as handle:
            pkl.dump(dict_results, handle)
elif execution == "oracle":
    with open('results_pkl/oracle.pkl', 'wb') as handle:
            pkl.dump(metric_res, handle)